In [ ]:
%load_ext autoreload

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

hv.extension('matplotlib')
import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath
import kwant 

import functools as ft

import hpc05
import sns_system, plotting_results
import topology, spectrum, scattering
import distributed_sns

In [ ]:
hpc05.kill_remote_ipcluster()

In [ ]:
client, dview, lview = hpc05.start_remote_and_connect(200, folder='~/two_dim_majoranas', timeout=180)

In [ ]:
vf = np.sqrt(2*1/params['m_eff'])
kf = vf * params['m_eff'] / params['hbar']
print(f'{1/kf:e}')

In [ ]:
kf*params['alpha_middle']

In [ ]:
vf = np.sqrt(2*params['mu']/params['m_eff'])
Eth = params['hbar'] * np.pi * vf / (2 * syst_pars['L_m'])

In [ ]:
Eth/(params['g_factor_middle']*params['mu_B'])

In [ ]:
params['m_eff'] * params['alpha_middle']**2 / params['hbar']**2

# Define system

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

syst_pars = {'L_down' :  1000,
               'L_up' : 1000,
               'L_m' : 250,
               'L_x' : 5,
               'a' :  5,
            'mu_from_bottom_of_spin_orbit_bands': True}

params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 1.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = 0.18,
                 Delta_right = 0.18,
                 B = 0.5,
                 phase = np.pi,
                 T = 0.025,
                 V = 0)

params = dict(**constants,
              **params_raw)

In [ ]:
locals().update(syst_pars)

# Make NS junction

In [ ]:
syst_ns = sns_system.make_ns_junction(**syst_pars)
_=kwant.plot(syst_ns)

# Make ASS

In [ ]:
%autoreload

In [ ]:
keys_with_bounds = {"mu": [0, 10],
                    "B" : [0, 2]}

metric_params_dict = {"transparency" : {},
                      "pfaffian" : {}}
ass = distributed_sns.AggregatesSimulationSet(keys_with_bounds,
                                              syst_pars, 
                                              params, 
                                              metric_params_dict)

In [ ]:
def _params_V(syst_pars, params, V):
    params['V'] = V
    return V

Vs = np.linspace(0, 6.0, 3)

params_V = [ft.partial(_params_V,
                           V=_V
                           ) for _V in Vs]

ass.add_dimension("transparency", params_V)

In [ ]:
def _params_phase(syst_pars, params, phase):
    params['phase'] = phase
    return phase

phases = np.linspace(0, np.pi, 3)

params_phase = [ft.partial(_params_phase,
                           phase=_phase
                           ) for _phase in phases]

ass.add_dimension("phase", params_phase)

In [ ]:
ass.make_balancing_learner(1000)
learner = ass.get_balancing_learner()

In [ ]:
runner = adaptive.Runner(learner, ntasks=1)#, executor=client)
runner.live_info(update_interval=1)

In [ ]:
runner.task.result()

In [ ]:
sum([l.npoints for l in learner.learners])

In [ ]:
kdims, pdict = ass.get_plot_dict(1000)
for k,v in pdict.items():
    pdict[k] = v[:,0:0.45]

In [ ]:
hv.extension('matplotlib')

In [ ]:
%%opts Image [colorbar=True aspect=1] {+framewise +axiswise}
hv.HoloMap(pdict, kdims=kdims)